einleitung
gesetzestexte ai act ...
forschungsfragen
ausarbeitung
fazit

## Versionsliste der genutzten Packages ##

In [19]:
%pip list

Package                                 Version
--------------------------------------- ------------
aiofiles                                24.1.0
aiohappyeyeballs                        2.4.3
aiohttp                                 3.10.10
aiosignal                               1.3.1
annotated-types                         0.7.0
anyio                                   4.6.2.post1
asttokens                               2.4.1
attrs                                   24.2.0
backoff                                 2.2.1
beautifulsoup4                          4.12.3
certifi                                 2024.8.30
cffi                                    1.17.1
chardet                                 5.2.0
charset-normalizer                      3.4.0
click                                   8.1.7
colorama                                0.4.6
comm                                    0.2.2
cryptography                            44.0.0
dataclasses-json                        0.6.7
debugpy 

In [20]:
#%pip install llama_index
#%pip install llama-index-llms-ollama
#%pip install llama-index-embeddings-huggingface

In [21]:
import os

import base64
import gc
import random
import tempfile
import time
import uuid

from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader


from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

from llama_index.core import SimpleDirectoryReader

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, Document

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

from llama_index.core.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter, SentenceSplitter

import re

from llama_index.core.schema import TextNode

import pandas as pd
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

from sentence_transformers import SentenceTransformer, util



Einlesen der Daten für die zusätzlichen Informationen für das RAG-System

In [22]:
def load_questions(input_file_path:str = "D:/LLama3/questions.csv"):
    df = pd.read_csv(input_file_path, sep=";", encoding='latin-1')
    return df

In [23]:
def load_documents(input_dir_path: str = "D:/LLama3/documents/"):
    loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".txt"],
            recursive=True
        )
    return loader.load_data()

In [24]:
def answer_user_question(useful_answers: list, index: VectorStoreIndex, question: str):
    #System-Message
    qa_chat_instr_new = (
#        """Du bist ein Anwalt mit 20 Jahren Berufserfahrung. Du stehst oft vor Gericht und bist regelmäßig in Gerichtsprozessen anwesend.
            """               
                Nutze die folgenden Informationen zur Beantwortung von Fragen.\n
                ---------------------\n
                {retrieved_graded_documents}\n
                ---------------------\n
                Gib eine präzise Antwort auf Basis der gegebenen Informationen.
    """
    )
                #Gebe immer folgende zusätzliche Informationen in der Antwort an:
                #Rechtliche Basis: [Hier rechtliche Informationen wie Gesetzestexte und Paragraphen angeben]
                #Besondere Informationen: [Hier zusätzliche Erläuterungen und Besonderheiten für die vorherigen Informationen angeben]
                #Genauer Wortlaut: [Hier den genauen Wortlaut aus dem Dokument einfügen]
                #Kurzantwort: [Hier kurz und knapp einfügen, was die wichtigsten Antworten & Fakten sind]

    qa_chat_instr_new = (
    #        """Du bist ein Anwalt mit 20 Jahren Berufserfahrung. Du stehst oft vor Gericht und bist regelmäßig in Gerichtsprozessen anwesend.
                """               
                    Beantworte die Frage mit Hilfe der Informationen aus dem hinzugefügten Kontext.\n
                """
                    #Gib eine präzise Antwort auf Basis der hier gegebenen Informationen.
                    #Wenn die Informationen keine Fakten bezüglich der Frage beinhalten, antworte mit 'Keine Informationen erhalten'
    )
    #User-Message
    qa_prompt_tmpl_str_new = (
                """
                Kontext:
                {retrieved_graded_documents}\n

                Frage: {query_str}\n"""
                )
    
    if len(useful_answers) > 0:
        concat_answers = ""
        for answ in useful_answers:
            concat_answers = concat_answers + answ + "\n\n"

        qa_chat_instr_new_formatted = str(qa_chat_instr_new).format(
            retrieved_graded_documents=concat_answers#.text#, query_str=question
        )
        qa_prompt_tmpl_str_new_formatted = str(qa_prompt_tmpl_str_new).format(
            query_str=question,
            retrieved_graded_documents=concat_answers#.text#, query_str=question
        )
    else:
        qa_chat_instr_new_formatted = str(qa_chat_instr_new).format(
            retrieved_graded_documents=""
        )

        qa_prompt_tmpl_str_new_formatted = str(qa_prompt_tmpl_str_new).format(
            query_str=question,
            retrieved_graded_documents=""#.text#, query_str=question

        )

    chat_text_qa_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=(
                qa_chat_instr_new_formatted
            ),
        ),
        ChatMessage(role=MessageRole.USER, content=qa_prompt_tmpl_str_new_formatted),
    ]
    
    text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

    query_engine_evaluator = index.as_query_engine(similarity_top_k=4, text_qa_template=text_qa_template)

    result = query_engine_evaluator.query(question)
    #print(result)
    return result

In [25]:
def retrieve_documents_from_vectorstore(question: str, index: VectorStoreIndex):
    #zuvor angelegter Index aus Dokumenten
    retriever = index.as_retriever(similarity_top_k=4)

    #Dokumente aus Retriever laden, welche Nähe/Ähnlichkeiten zur Frage haben
    retr_answers = retriever.retrieve(question)
    return retr_answers

In [26]:
def retrieve_documents_from_vectorstore_new(question: str, index: VectorStoreIndex):
    top_k = 4

    # configure retriever
    retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
    )

    # assemble query engine
    query_engine = RetrieverQueryEngine(        
        retriever=retriever,
        node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
    )

    response = query_engine.query(question)
    # reformat response
    retr_answers = []
    #for i in range(top_k):
    for i in range(len(response.source_nodes)):
        retr_answers.append(response.source_nodes[i].text)
        
    return retr_answers

In [27]:
def retrieve_documents_from_vectorstore_new2(question: str, index_base_docs: list, embedding_model: SentenceTransformer, doc_emb):
    newtopk = 4

    #Encode query and documents
    #print("embedding question and docs")
    query_emb = embedding_model.encode(question)
    

    #Compute dot score between query and all document embeddings
    #print("computing scores")
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

    #Combine docs & scores
    #print("combining docs & scores")
    doc_score_pairs = list(zip(index_base_docs, scores))

    #Sort by decreasing score
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

    #Output passages & scores
    #for doc, score in doc_score_pairs:
    #    print(score, doc)

    top_k_doc_score_pairs = doc_score_pairs[:newtopk]

    top_k_docs = []
    for doc_score in top_k_doc_score_pairs:
        top_k_docs.append(doc_score[0].text)

    print(top_k_docs[0])
    return top_k_docs

In [28]:
def grade_retrieved_documents(question:str, retr_answers: list, index: VectorStoreIndex):
    doc_grader_instr_new = """Du bewertest die Relevanz von einem abgerufenen Dokument für die Frage eines Benutzers. 
    Bewerte sorgfältig und objektiv, ob das Dokument Stichworte oder Informationen enthält, die einen Bezug zu der Frage haben.
    Gib eine Antwort im JSON Format, welche den Wert 'ja' oder 'nein' enthält.
    Wenn das Dokument Stichworte oder Informationen mit Bezug zur Frage enthält, antworte mit 'ja', sonst mit 'nein'.
    """

    #User-Message
    doc_grader_prompt_new = """Abgerufenes Dokument: \n\n {document} \n\n. Frage: \n\n {question}."""
    
    #für Antwort in json-Form wird llm_json für Anfragen verwendet
    #Settings.llm = llm_json # specifying the llm to be used

    #print(len(retr_answers))

    #Da die Antworten nicht immer tatsächlich relevant sind bzw. Hallucinations beinhalten müssen diese erneut evaluiert werden. 
    #Hierfür wird dem LLM gesagt, dass es die erhaltenen Dokumente hinsichtlich der Fragestellung erneut bewerten soll.
    useful_answers = []
    for doc in retr_answers:
        #print(doc.text)
        doc_grader_prompt_formatted = str(doc_grader_prompt_new).format(
        document=doc.text, question=question
        )

        # Text QA Prompt
        chat_text_qa_msgs = [
            ChatMessage(
                role=MessageRole.SYSTEM,
                content=(
                    doc_grader_instr_new
                ),
            ),
            ChatMessage(role=MessageRole.USER, content=doc_grader_prompt_formatted),
        ]
        
        text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

        query_engine_doc_grader = index.as_query_engine(similarity_top_k=3, text_qa_template=text_qa_template)

        result = query_engine_doc_grader.query(question)
        if 'ja' in str(result).lower():
            useful_answers.append(doc.text)
            print(result)
            print(doc.text)

    return(useful_answers)


In [29]:
def grade_retrieved_documents_new(question:str, retr_answers: list, index: VectorStoreIndex):
    doc_grader_instr_new = """Du bewertest die Relevanz von einem abgerufenen Dokument für die Frage eines Benutzers. 
    Bewerte sorgfältig und objektiv, ob das Dokument Stichworte oder Informationen enthält, die einen Bezug zu der Frage haben.
    Gib eine Antwort im JSON Format, welche den Wert 'ja' oder 'nein' enthält.
    Wenn das Dokument Stichworte oder Informationen mit Bezug zur Frage enthält, antworte mit 'ja', sonst mit 'nein'.
    """

    #User-Message
    doc_grader_prompt_new = """Abgerufenes Dokument: \n\n {document} \n\n. Frage: \n\n {question}."""
    
    #für Antwort in json-Form wird llm_json für Anfragen verwendet
    #Settings.llm = llm_json # specifying the llm to be used

    #print(len(retr_answers))

    #Da die Antworten nicht immer tatsächlich relevant sind bzw. Hallucinations beinhalten müssen diese erneut evaluiert werden. 
    #Hierfür wird dem LLM gesagt, dass es die erhaltenen Dokumente hinsichtlich der Fragestellung erneut bewerten soll.
    useful_answers = []
    for answer in retr_answers:
        #print(doc.text)
        doc_grader_prompt_formatted = str(doc_grader_prompt_new).format(
        document=answer, question=question
        )

        # Text QA Prompt
        chat_text_qa_msgs = [
            ChatMessage(
                role=MessageRole.SYSTEM,
                content=(
                    doc_grader_instr_new
                ),
            ),
            ChatMessage(role=MessageRole.USER, content=doc_grader_prompt_formatted),
        ]
        
        text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

        query_engine_doc_grader = index.as_query_engine(similarity_top_k=3, text_qa_template=text_qa_template)

        result = query_engine_doc_grader.query(question)
        if 'ja' in str(result).lower():
            useful_answers.append(answer)
            print(result)
            print(answer)


    #Wieder auf Volltext-LLM ändern
    #Settings.llm = llm # specifying the llm to be used
    return(useful_answers)


Dokumente aus Verzeichnis laden

In [30]:
questions = load_questions()
docs = load_documents()

Einstellungen/Vorbereitung der Dokumente für Indexierung

In [31]:
#nodeparser test
#node_parser = SentenceSplitter(secondary_chunking_regex="\r\n*Artikel \d*\r\n")
# use transforms directly

#nodes = node_parser(docs)
text_chunks= []
index_base_docs = []

#gg_nodes = []
#gg_docs = []
for doc in docs:
    if "Grundgesetz - Kopie.txt" in doc.metadata["file_path"] or "Grundgesetz.txt" in doc.metadata["file_path"]:
        #text_chunks = re.split(r'Artikel \d', doc.text)
        text_chunks = re.split('(?s)(\nArtikel [0-9]+.*?(?=\nArtikel))', doc.text)
        for chunk in text_chunks:
            #print(chunk)
            if not(chunk == "" or not chunk):
                #chunk_clean = chunk.replace(' ', ' ').replace('\t', ' ').replace('\n', ' ').replace('\r', ' ')
                #chunk_clean = str(re.sub(r"\s+", " ", chunk))
                #print(chunk_clean)
                #gg_nodes.append(TextNode(text=chunk))
                #gg_docs.append(Document(text=chunk))
                index_base_docs.append(Document(text=chunk))
        #print(gg_nodes)
    else:
        index_base_docs.append(doc)

#cleaning tab, newline, whitespaces
for doc in index_base_docs:
    doc.text = str(re.sub(r"\s+", " ", doc.text))



Setup des Embedding Models, welches für die Vektorisierung der Daten genutzt wird und Durchführung der Indexierung der eingelesenen Dokumente mit dem Embedding Model

In [32]:
#embed_model = HuggingFaceEmbedding( model_name="sentence-transformers/all-MiniLM-L6-v2", trust_remote_code=True)
##embed_model = HuggingFaceEmbedding( model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1", trust_remote_code=True)


embed_model = HuggingFaceEmbedding( model_name="nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
#embed_model = HuggingFaceEmbedding( model_name="sentence-transformers/msmarco-distilbert-base-tas-b", trust_remote_code=True)

# ====== Create vector store and upload indexed data ======
Settings.embed_model = embed_model # we specify the embedding model to be used
#index_documents = VectorStoreIndex.from_documents(docs, show_progress=True, transformations=my_transformations)
index = VectorStoreIndex.from_documents(index_base_docs, show_progress=True)

<All keys matched successfully>
Generating embeddings: 100%|██████████| 11/11 [00:09<00:00,  1.13it/s]


Ollama Models für Analyse & Vergleich 

Vergleich der Antworten mit und ohne zusätzlichen Informationen aus RAG für die folgenden Modelle

In [33]:
models = ["llama3.2:1b", "llama3.2:3b", "llama3.1:8b"]

Setup des LLM-Models, welches die Tatsächliche Anfrage beantwortet.

Vorbereitung der Query-Engine, welche dem LLM Zugriff auf den Vector Store gewährt.

In [34]:
q_model_index = 2
current_model = models[q_model_index]
# setting up the llm
llm = Ollama(model=current_model, request_timeout=120.0)
llm_json = Ollama(model=current_model, request_timeout=120.0, json_mode=True) 

# ====== Setup a query engine on the index previously created ======
Settings.llm = llm # specifying the llm to be used


In [35]:
questions_models = pd.DataFrame(questions.columns.values).tail(-1)
for questions_models_index, questions_model in questions_models.iterrows():
    print(questions_model[0])

llama3.2:1b
llama3.2:3b
llama3.1:8b


Retrieval Grader   

In [36]:
#Frage, die beantwortet werden sol
#question = "Was ist Notwehr?"
#question = "Was genau steht in Artikel 1 des Grundgesetzes?"
#question = "Mit Rakete zu spät?"
answers = []
answers_new = []

st_embedding_model = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')

newdocs = []
for doc in index_base_docs:
    newdocs.append(doc.text)
st_doc_emb = st_embedding_model.encode(newdocs)

for questions_models_index, questions_model in questions_models.iterrows():
    current_model = questions_model[0]
    print("Question Model Index: " + str(questions_models_index) + " Model: " + str(current_model))
    
    # setting up the llm
    llm = Ollama(model=current_model, request_timeout=120.0)
    llm_json = Ollama(model=current_model, request_timeout=120.0, json_mode=True) 
    Settings.llm = llm # specifying the llm to be used

    for questions_index, questions_row in questions.iterrows():
        print("Question Index: " + str(questions_index))
        df_question = str(questions_row['Frage'])
        #print(df_question)
        #Abfrage an Vectorstore um Dokumente mit Ähnlichkeit zur Frage zu finden
        #retrieved_answers = retrieve_documents_from_vectorstore(df_question, index)
        retrieved_answers_new = retrieve_documents_from_vectorstore_new(df_question, index)
        retrieved_answers_new2 = retrieve_documents_from_vectorstore_new2(df_question, index_base_docs, st_embedding_model, st_doc_emb)
        print("Retrieved Answers: " + str(len(retrieved_answers_new2)))

        #Bewertung der erhaltenen Dokumente hinsichtlich tatsächlicher Relevanz
        Settings.llm = llm_json # specifying the llm to be used
        #useful_answers = grade_retrieved_documents(df_question, retrieved_answers, index)
        useful_answers_new = grade_retrieved_documents_new(df_question, retrieved_answers_new2, index)
        print("Useful Answers: " + str(len(useful_answers_new)))

        Settings.llm = llm # specifying the llm to be used
        #letzter Aufruf, der die Antwort für den User zurückgibt
        #answer = str(answer_user_question(useful_answers=useful_answers, index=index, question=df_question))
        answer_new = str(answer_user_question(useful_answers=useful_answers_new, index=index, question=df_question))

        #answers.append(answer)
        answers_new.append(answer_new)
        questions.iloc[questions_index, questions_models_index] = answer_new



Question Model Index: 1 Model: llama3.2:1b
Question Index: 0
Strafgesetzbuch Allgemeiner Teil (§§ 1 - 79b) 2. Abschnitt - Die Tat (§§ 13 - 37) 4. Titel - Notwehr und Notstand (§§ 32 - 35) § 32 Notwehr (1) Wer eine Tat begeht, die durch Notwehr geboten ist, handelt nicht rechtswidrig. (2) Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.
Retrieved Answers: 4
{
  "Relevanz": "ja",
  "Antwort": {
    "Notwehr": "Ja"
  }
}
Strafgesetzbuch Allgemeiner Teil (§§ 1 - 79b) 2. Abschnitt - Die Tat (§§ 13 - 37) 4. Titel - Notwehr und Notstand (§§ 32 - 35) § 32 Notwehr (1) Wer eine Tat begeht, die durch Notwehr geboten ist, handelt nicht rechtswidrig. (2) Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.
{
  "Notwendigkeit": "ja",
  "Geringkeit": "nein"
}
Fall Nr. 1234 Anklagepunkt: Mit Superbahn zu spät Urteil: Schuldig Stra

C:\Users\doein\AppData\Local\Temp\ipykernel_9912\1462259441.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Beantwort:

Noch keine Antwort, da die Frage nicht direkt im Zusammenhang mit den Informationen zum Strafgesetzbuch und dem Grundgesetz steht.

Allerdings kann aus den bereitgestellten Informationen folgern, dass Notwehr eine Rechtsposition bezeichnet ist, bei der jemand handelt, um gegen einen gegenwärtigen rechtswidrigen Angriff zu vorgehen oder sich selbst abzuwehren. 

In Bezug auf das Strafgesetzbuch Allgemeiner Teil und das Grundgesetz für die Bundesrepublik Deutschland könnten Notwehr im Kontext von Straftaten, wie z.B. "Mit Superbahn zu spät" (Anklagepunkt: Schuldig Strafmaß), eine Rechtsposition darstellen, bei der jemand handelt, um sich selbst oder einem anderen abzuwehren. 

Es ist jedoch ratsam, die spezifischen rechtlichen Aspekte und Kontexte jedes Falls zu berücksichtigen, um e

Question Index: 1
Grundgesetz für die Bundesrepublik Deutschland Vom 23. Mai 1949 (BGBl. S. 1) Zuletzt geändert durch das Gesetz zur Änderung des Grundgesetzes (Artikel 96) vom 26. 7. 2002 (BGBl. I S. 2863) Artikel 1 (1) Die Würde des Menschen ist unantastbar. Sie zu achten und zu schützen, ist Verpflichtung aller staatlichen Gewalt. (2) Das Deutsche Volk bekennt sich darum zu unverletzlichen und unveräußerlichen Menschenrechten als Grundlage jeder menschlichen Gemeinschaft, des Friedens und der Gerechtigkeit in der Welt. (3) Die nachfolgenden Grundrechte binden Gesetzgebung, vollziehende Gewalt und Rechtsprechung als unmittelbar geltendes Recht.
Retrieved Answers: 4
{
  "beantwortung": "Ja"
}
Grundgesetz für die Bundesrepublik Deutschland Vom 23. Mai 1949 (BGBl. S. 1) Zuletzt geändert durch das Gesetz zur Änderung des Grundgesetzes (Artikel 96) vom 26. 7. 2002 (BGBl. I S. 2863) Artikel 1 (1) Die Würde des Menschen ist unantastbar. Sie zu achten und zu schützen, ist Verpflichtung aller

C:\Users\doein\AppData\Local\Temp\ipykernel_9912\1462259441.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Nach § 32 des Strafgesetzbuches handelt eine Person nicht rechtswidrig, wenn sie eine Tat begeht, die durch Notwehr geboten ist. Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  questions.iloc[questions_index, questions_models_index] = answer_new


Question Index: 1
Grundgesetz für die Bundesrepublik Deutschland Vom 23. Mai 1949 (BGBl. S. 1) Zuletzt geändert durch das Gesetz zur Änderung des Grundgesetzes (Artikel 96) vom 26. 7. 2002 (BGBl. I S. 2863) Artikel 1 (1) Die Würde des Menschen ist unantastbar. Sie zu achten und zu schützen, ist Verpflichtung aller staatlichen Gewalt. (2) Das Deutsche Volk bekennt sich darum zu unverletzlichen und unveräußerlichen Menschenrechten als Grundlage jeder menschlichen Gemeinschaft, des Friedens und der Gerechtigkeit in der Welt. (3) Die nachfolgenden Grundrechte binden Gesetzgebung, vollziehende Gewalt und Rechtsprechung als unmittelbar geltendes Recht.
Retrieved Answers: 4
Useful Answers: 0
Question Index: 2
Mein Statement zum Abschlusskolloquium: Der erste Transferbericht zeigt, wie verschiedene Themengebiete aus dem Cloud-Umfeld, welches für die Big Data Architektur genutzt wird, auf konventionelle, nicht Cloud-native Anwendungen angewendet werden kann und dort zu Vorteilen und Nutzen führ

C:\Users\doein\AppData\Local\Temp\ipykernel_9912\1462259441.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Die Antwort auf die Frage lautet:

Notwehr ist die Verteidigung, die erforderlich ist, um einen gegenwärtigen rechtswidrigen Angriff von sich oder einem anderen abzuwenden.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  questions.iloc[questions_index, questions_models_index] = answer_new


Question Index: 1
Grundgesetz für die Bundesrepublik Deutschland Vom 23. Mai 1949 (BGBl. S. 1) Zuletzt geändert durch das Gesetz zur Änderung des Grundgesetzes (Artikel 96) vom 26. 7. 2002 (BGBl. I S. 2863) Artikel 1 (1) Die Würde des Menschen ist unantastbar. Sie zu achten und zu schützen, ist Verpflichtung aller staatlichen Gewalt. (2) Das Deutsche Volk bekennt sich darum zu unverletzlichen und unveräußerlichen Menschenrechten als Grundlage jeder menschlichen Gemeinschaft, des Friedens und der Gerechtigkeit in der Welt. (3) Die nachfolgenden Grundrechte binden Gesetzgebung, vollziehende Gewalt und Rechtsprechung als unmittelbar geltendes Recht.
Retrieved Answers: 4
{
  "relevant": "ja"
}
Grundgesetz für die Bundesrepublik Deutschland Vom 23. Mai 1949 (BGBl. S. 1) Zuletzt geändert durch das Gesetz zur Änderung des Grundgesetzes (Artikel 96) vom 26. 7. 2002 (BGBl. I S. 2863) Artikel 1 (1) Die Würde des Menschen ist unantastbar. Sie zu achten und zu schützen, ist Verpflichtung aller sta

BERT Test Model

In [37]:
from transformers import AutoTokenizer, AutoModel
bert_model_name = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = AutoModel.from_pretrained(bert_model_name)

In [38]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.2050, -0.2159,  0.3971,  ...,  0.4766,  0.1580,  0.1503],
         [ 0.3262, -0.3483,  0.5030,  ...,  0.1136,  0.5199,  0.3270],
         [-0.4163,  0.1461,  0.0341,  ...,  0.2915,  0.2132,  0.6478],
         ...,
         [ 0.6331,  0.2565,  0.1538,  ...,  0.0307, -0.1560, -0.2545],
         [-0.1826, -0.2991,  0.0837,  ..., -0.1916, -0.1148, -0.1778],
         [-0.2082, -0.6072, -0.2180,  ..., -0.2545,  0.0786,  0.3093]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-7.2543e-01,  1.6392e-01,  1.0000e+00,  2.9478e-01,  2.5463e-01,
          8.3638e-02, -9.9495e-01, -4.5405e-01,  1.0000e+00,  9.9985e-01,
          1.0000e+00, -8.6434e-03,  8.1573e-02, -1.0000e+00,  3.4033e-01,
          3.9244e-01,  8.7590e-01,  3.6538e-02,  4.2574e-03,  9.8924e-01,
          5.5207e-02, -9.9675e-01,  1.0000e+00,  1.5514e-01, -2.1364e-01,
         -9.9957e-01, -1.1825e-01, -9.9883e-01, -7.5258e-02, -9.832

In [39]:
from transformers import AutoModelForQuestionAnswering

question_model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
bert_question = "How many programming languages does BLOOM support?"
bert_context = "BLOOM has 176 billion parameters and can generate text in 46 natural languages and 13 programming languages."

In [41]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model=bert_model_name)
question_answerer(question=bert_question, context=bert_context)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.003883825149387121,
 'start': 0,
 'end': 54,
 'answer': 'BLOOM has 176 billion parameters and can generate text'}

In [42]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
inputs = tokenizer(bert_question, bert_context, return_tensors="pt")

In [43]:
import torch
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)
with torch.no_grad():
    outputs = model(**inputs)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [45]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'does bloom support? [SEP]'